In [1]:
import time
import numpy as np
import pandas as pd
from amuse.lab import Particles
from amuse.couple import bridge
import matplotlib.pyplot as plt
from amuse.units.constants import G
from amuse.units import units, constants
from amuse.lab import Huayno, nbody_system
from amuse.community.ph4.interface import ph4
from amuse.ext.orbital_elements import get_orbital_elements_from_binary
from amuse.ext.orbital_elements import new_binary_from_orbital_elements

In [6]:
test_particles = Particles(3)
test_particles[0].position = [0,0,0] | units.m
test_particles[0].velocity = [0,0,0] | units.m/units.s
test_particles[1].position = [1,0,0] | units.m
test_particles[1].velocity = [0,2,0] | units.m/units.s
test_particles[2].position = [0,3,0] | units.m
test_particles[2].velocity = [-1,0,0] | units.m/units.s
for i in range(3):
    test_particles[i].mass = 1 |units.kg 
    
E_kin = test_particles.kinetic_energy()
E_pot = test_particles.potential_energy()

test_particles.mass.value_in(units.kg)
E_kin

quantity<2.5 m**2 * kg * s**-2>

In [17]:
#grav = Huayno()
#grav.add_particles(test_particles)
grav.stop()

In [5]:
a = [0,1,2,3]
b = [2,3,4,5]
c = [2,5,6,2]

df = pd.DataFrame({'a':a,
                   'b':b,
                   'c':c})
df

,a,b,c
0,0,2,2
1,1,3,5
2,2,4,6
3,3,5,2


In [10]:
end = 10|units.yr
dt = 1|units.yr
times = np.arange(0,end.value_in(units.yr),dt.value_in(units.yr))*1|units.yr
times

quantity<[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] yr>

In [31]:
def energy_dissipation(system,k=0.565,alpha=0.1):
    """ Calculates the total energy dissipation due to the tidal force for a particle system. The second
    particle in the particle set is the particle that exerts the tidal force."""
    #defining some constants
    R = (1|units.RJupiter).value_in(units.m) #radius of Jupiter
    M = (1|units.MJupiter).value_in(units.kg)
    G = constants.G.value_in( (units.m)**3 / (units.kg * units.s**2) )
    
    #getting some parameters/values from the particle set
    m = system.mass.value_in(units.kg)
    r = np.sqrt(np.sum(np.square(system.position.value_in(units.m) - \
                                 system[1].position.value_in(units.m)), axis=1))
    
    #to exclude the force of Jupiter on itself
    mask = np.arange(len(system)) != 1
    
    #the energy dissipation
    dE_dt = -9*k*G**(3/2)*M**(1/2)*m[mask]**2*R**5*np.sin(2*alpha)/(2*r[mask]**(15/2))
    
    return np.sum(dE_dt) * 1|units.J/units.s

def get_energies(system,timestep,k=0.565,alpha=0.1,):
    """ Calculates the total energy of the system, along with the energy dissipation."""
    #getting the kinetic and potential energy from the particle system
    E_kin = system.kinetic_energy()
    E_pot = system.potential_energy()
    E_tot = E_kin + E_pot
    
    #getting the energy dissipation
    dE_dt = energy_dissipation(system)
    #getting the total dissipated energy in the following timestep
    dE = dE_dt*timestep
    
    return E_tot, dE

#get_energies(test_particles,10|units.s)
help(energy_dissipation)

Help on function energy_dissipation in module __main__:

energy_dissipation(system, k=0.565, alpha=0.1)
    Calculates the total energy dissipation due to the tidal force for a particle system. The second
    particle in the particle set is the particle that exerts the tidal force.

